In [ ]:
!python -m pip install -r requirements.txt

In [ ]:
from math import cos, sin
import numpy as np

L = np.array([100.0, 100.0])
T = 10.0
O, AB = np.array([40.0, 50.0]), np.array([7.0, 11.0])
WIDTH = np.array([1.0, 2.0])
(AMPLITUDE, GAMMA) = 1.0, 0.25

def condition(u: np.array, k: int) -> type(None):
    t = T / len(u) * k
    xy = O + AB * np.array([cos(t), sin(t)])
    bounds = u.shape[1:] / L * (np.outer(np.array([-1, 1]), WIDTH) + xy)
    i, j = bounds.round().astype(int).T
    u[k, i[0]:i[1], j[0]:j[1]] = AMPLITUDE

def calculate(u: np.ndarray) -> np.ndarray:
    for k in range(u.shape[0] - 1):
        A = u[k, 2:  , 1:-1]
        B = u[k,  :-2, 1:-1]
        C = u[k, 1:-1, 2:  ]
        D = u[k, 1:-1,  :-2]
        E = u[k, 1:-1, 1:-1]
        u[k + 1, 1:-1, 1:-1] = GAMMA * (A + B + C + D - 4.0 * E) + E
        condition(u, k + 1)
    return u

In [ ]:
u = np.zeros((256, 512, 512))
condition(u, 0)
u = calculate(u)
vmin, vmax = u.min(), u.max()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

figure, axes = plt.subplots(figsize=(16, 9), dpi=400)

def animate(k: int):
    print(F'animate({k:03})', end='\r')
    axes.cla()
    axes.pcolormesh(u[k], cmap=plt.cm.jet, vmin=vmin, vmax=vmax)
    axes.set_title(F"t = {T / len(u) * k}")
    axes.set_xlabel('x')
    axes.set_ylabel('y')
    return axes

In [ ]:
interval = 1000.0 * T / len(u)
animation = FuncAnimation(figure, animate, interval=interval, frames=len(u), repeat=False)
animation.save("animation.gif", writer='pillow')